# CatBoost with 18 Features (Spange + Kinetics)

**Problem**: Top 10 features (exp_031) underfitted (CV 0.009984). 145 features might overfit.
**Hypothesis**: 18 features (13 Spange + 4 Kinetics + 1 SolventB%) is the sweet spot.
**Approach**:
- Features: All 13 Spange descriptors + 4 Kinetics + SolventB%
- Model: CatBoostRegressor (RMSE loss, depth=6, lr=0.05, iterations=1000)
- Post-processing: Clip predictions to [0, 1]
- Validation: LOO (Single) + LORO (Full)

**Baseline**: exp_026 CV 0.008465, exp_031 CV 0.009984

In [1]:
# Standard imports
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
        # Add SolventB% column for single solvent (all zeros)
        X["SolventB%"] = 0.0
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [2]:
# Load feature lookups - ONLY Spange needed
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)

print(f'Spange: {SPANGE_DF.shape}')
print('Using 18 features: 13 Spange + 4 Kinetics + 1 SolventB%')

Spange: (26, 13)
Using 18 features: 13 Spange + 4 Kinetics + 1 SolventB%


In [3]:
# Feature Engineering for 18 Features
class Featurizer18:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        # 18 features: 4 Kinetics + 13 Spange + 1 SolventB%
        self.feats_dim = 4 + 13 + 1

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        time_m = X_vals[:, 0:1]
        temp_c = X_vals[:, 1:2]
        
        # Kinetic features (4)
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        
        # [Time, Temp, 1000/T, 1000/T*ln(t)]
        X_kinetic = np.hstack([time_m, temp_c, inv_temp, interaction])
        
        # SolventB% (1)
        if "SolventB%" in X.columns:
            pct = X["SolventB%"].values.reshape(-1, 1)
        else:
            pct = np.zeros((len(X), 1))
            
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            
            if flip:
                # Flip: A becomes B, B becomes A, pct becomes 1-pct
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                # SolventB% for flipped is 1-pct? No, usually we just swap A and B and keep pct as "percentage of second solvent"
                # But here we are mixing features linearly.
                # Standard mixing: A*(1-pct) + B*pct
                # Flipped mixing: B*(1-(1-pct)) + A*(1-pct) = B*pct + A*(1-pct) -> SAME!
                # Wait, if we flip, we are saying Solvent A is now the one that was Solvent B.
                # So the new "SolventB%" is the percentage of the OLD Solvent A.
                # If original was 10% B, then flipped is 90% A (which is now 'B').
                # So pct_flipped = 1 - pct.
                pct_feat = 1 - pct
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                pct_feat = pct
        else:
            # Single solvent
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            pct_feat = np.zeros((len(X), 1))
        
        return np.hstack([X_kinetic, X_spange, pct_feat])

print(f'Featurizer18 defined with {Featurizer18().feats_dim} features')

Featurizer18 defined with 18 features


In [4]:
# CatBoost Wrapper
class CatBoostWrapper:
    def __init__(self, data='single'):
        self.data_type = data
        self.featurizer = Featurizer18(mixed=(data=='full'))
        self.models = []

    def train_model(self, X_train, y_train):
        X_std = self.featurizer.featurize(X_train, flip=False)
        y_vals = y_train.values
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = np.vstack([X_std, X_flip])
            y_all = np.vstack([y_vals, y_vals])
        else:
            X_all, y_all = X_std, y_vals
        
        self.models = []
        # Train separate model for each target
        for i in range(3):
            model = CatBoostRegressor(
                iterations=1000, 
                learning_rate=0.05, 
                depth=6, 
                loss_function='RMSE',
                random_seed=42, 
                verbose=0,
                allow_writing_files=False
            )
            model.fit(X_all, y_all[:, i])
            self.models.append(model)

    def predict(self, X_test):
        X_feat = self.featurizer.featurize(X_test, flip=False)
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_test, flip=True)
        
        preds = []
        for i, model in enumerate(self.models):
            pred = model.predict(X_feat)
            
            if self.data_type == 'full':
                pred_flip = model.predict(X_flip)
                pred = (pred + pred_flip) / 2
            
            # Clip predictions to [0, 1]
            pred = np.clip(pred, 0, 1)
            preds.append(pred)
        
        return np.column_stack(preds)

print('CatBoostWrapper defined')

CatBoostWrapper defined


In [5]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = CatBoostWrapper(data='single')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:01,  1.05s/it]

2it [00:02,  1.02s/it]

3it [00:03,  1.02s/it]

4it [00:04,  1.00s/it]

...

21it [00:20,  1.01it/s]

22it [00:21,  1.01it/s]

23it [00:22,  1.01it/s]

24it [00:23,  1.00it/s]

24it [00:23,  1.00it/s]

In [6]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = CatBoostWrapper(data='full')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:01,  1.97s/it]

2it [00:03,  1.98s/it]

3it [00:06,  2.01s/it]

4it [00:07,  1.99s/it]

...

10it [00:19,  2.00s/it]

11it [00:21,  2.01s/it]

12it [00:24,  2.03s/it]

13it [00:26,  2.04s/it]

13it [00:26,  2.01s/it]

In [7]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [8]:
# Calculate CV score (for verification only - NOT part of submission)
X_single, Y_single = load_data("single_solvent")
X_full, Y_full = load_data("full")

# Get actuals in same order as predictions
actuals_single = []
for solvent in sorted(X_single["SOLVENT NAME"].unique()):
    mask = X_single["SOLVENT NAME"] == solvent
    actuals_single.append(Y_single[mask].values)
actuals_single = np.vstack(actuals_single)

actuals_full = []
ramps = X_full[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
for _, row in ramps.iterrows():
    mask = (X_full["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X_full["SOLVENT B NAME"] == row["SOLVENT B NAME"])
    actuals_full.append(Y_full[mask].values)
actuals_full = np.vstack(actuals_full)

# Get predictions
preds_single = submission_single_solvent[['target_1', 'target_2', 'target_3']].values
preds_full = submission_full_data[['target_1', 'target_2', 'target_3']].values

# Calculate MSE
mse_single = np.mean((actuals_single - preds_single) ** 2)
mse_full = np.mean((actuals_full - preds_full) ** 2)
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== CV SCORE VERIFICATION ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nexp_026 baseline (MLP+LGBM): CV 0.008465')
print(f'exp_031 (CatBoost Top 10): CV 0.009984')
print(f'This (CatBoost 18 feats): CV {overall_mse:.6f}')

if overall_mse < 0.008465:
    improvement = (0.008465 - overall_mse) / 0.008465 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than exp_026!')
else:
    degradation = (overall_mse - 0.008465) / 0.008465 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than exp_026')


=== CV SCORE VERIFICATION ===
Single Solvent MSE: 0.011295 (n=656)
Full Data MSE: 0.010816 (n=1227)
Overall MSE: 0.010983

exp_026 baseline (MLP+LGBM): CV 0.008465
exp_031 (CatBoost Top 10): CV 0.009984
This (CatBoost 18 feats): CV 0.010983

✗ WORSE: 29.75% worse than exp_026
